In [1]:
from sklearn import svm
# from project.dg_face import utils, detection, alignment, recognition
import os
import numpy as np
from sklearn.preprocessing import Normalizer, LabelEncoder

In [2]:
train = np.load("npz_30_20/train.npz")
X_train_raw, y_train_raw = train['X'], train['y']
test = np.load('npz_30_20/test.npz')
X_test_raw, y_test_raw = test['X'], test['y']

In [17]:
in_encoder = Normalizer(norm='l2')
X_train = in_encoder.transform(X_train_raw)
X_test = in_encoder.transform(X_test_raw)
# X_train = X_train_raw
# X_test = X_test_raw

In [4]:
out_encoder = LabelEncoder()
y_train = out_encoder.fit_transform(y_train_raw)

In [21]:
def score(model, threshold, *, verbose=False):
    mistakes = []
    unknown_probas, known_probas = [], []
    for (x, y) in zip(X_test, y_test_raw):
        probas = model.predict_proba([x])[0]
        pred = probas.argmax()
        proba = probas[pred]
        are_matches = proba * 100 >= threshold
        label = out_encoder.inverse_transform([pred])[0] if are_matches else "unknown"
        if verbose:
            print("actually:", y, "predicted:", label, "proba:", proba)
        if y == 'unknown':
            unknown_probas.append(proba)
        else:
            known_probas.append(proba)
        if label != y:
            mistakes.append([y, label, proba])
    print("mistake count", len(mistakes))
    known_mean = 0 if len(known_probas) == 0 else np.array(known_probas).mean()
    unknown_mean = 0 if len(unknown_probas) == 0 else np.array(unknown_probas).mean()
    print("known mean", known_mean, "unknown mean", unknown_mean, "delta", abs(known_mean - unknown_mean))
    return len(mistakes)

In [ ]:
def score(model, threshold, *, verbose=False):
    mistakes = []
    for (x, y) in zip(X_test, y_test_raw):
        probas = model.predict_proba([x])[0]
        pred = probas.argmax()
        proba = probas[pred]
        are_matches = proba * 100 >= threshold
        label = out_encoder.inverse_transform([pred])[0] if are_matches else "unknown"
        if verbose:
            print("actually:", y, "predicted:", label, "proba:", proba)
        if label != y:
            mistakes.append([y, label, proba])
    print("mistake count", len(mistakes))
    return mistakes

Создаем модель для извлечения признаков

In [183]:
from sklearn.calibration import CalibratedClassifierCV

In [22]:
# clf =    CalibratedClassifierCV(svm.SVC(kernel='rbf', probability=True, C=20, random_state=10), cv=3)

# clf = svm.SVC(kernel='linear', probability=True, C=50, random_state=10)
clf = svm.SVC(kernel='rbf', probability=True, C=50, random_state=10)
clf.fit(X_train, y_train)
score(clf, 11.5, verbose=True)
# clfs = [
#     svm.SVC(kernel='linear', probability=True, C=50, random_state=10),
#     svm.SVC(kernel='rbf', probability=True, C=50, random_state=10),
    # svm.SVC(kernel='poly', degree=5, probability=True, C=50, random_state=10),
# ]
# thrs = [
#     11.5,
#     11.5,
#     11,
#     11,
# ]

# for clf, threshold in zip(clfs, thrs):
#     clf.fit(X_train, y_train)
#     score(clf, threshold, verbose=True)
    # score(clf, threshold)

actually: unknown predicted: unknown proba: 0.10654072865134827
actually: unknown predicted: unknown proba: 0.07740878866781184
actually: unknown predicted: unknown proba: 0.07685084104795382
actually: unknown predicted: unknown proba: 0.06611585106118525
actually: unknown predicted: unknown proba: 0.06846459973497107
actually: unknown predicted: unknown proba: 0.07526176486536472
actually: unknown predicted: unknown proba: 0.08373541672960777
actually: unknown predicted: unknown proba: 0.08127657333048073
actually: unknown predicted: unknown proba: 0.0912207241417642
actually: unknown predicted: unknown proba: 0.10727479479002794
actually: unknown predicted: unknown proba: 0.08951890012304287
actually: unknown predicted: unknown proba: 0.10977032148227929
actually: unknown predicted: unknown proba: 0.09857598760005648
actually: unknown predicted: unknown proba: 0.07175737827753817
actually: unknown predicted: unknown proba: 0.09153946136449298
actually: unknown predicted: unknown prob

0

Попробовать нарисовать 

In [ ]:
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

In [ ]:
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_train)
plt.scatter(X_pca[:,0], X_pca[:,1], c=y_train, cmap='inferno', marker='x')
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.title('SVC Classification')
plt.show()